In [ ]:
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

from pathlib import Path
path = Path('data')

In [ ]:
import geopandas


countries = geopandas.read_file(
    f'zip://{path.joinpath("ne_110m_admin_0_countries.zip")}'
)

rivers = geopandas.read_file(
    f'zip://{path.joinpath("ne_50m_rivers_lake_centerlines.zip")}'
)

In [ ]:
countries.head()

In [ ]:
ax = countries.plot(figsize=(20, 20))

<div class="alert alert-success" style="font-size:120%">
<b>Exercise</b>: <br><br>

What the type is of `ax` object here?
</div>

In [ ]:
amazon = rivers[rivers['name'] == 'Amazonas'].geometry.squeeze()

amazon

<div class="alert alert-success" style="font-size:120%">
<b>Exercise</b>: <br><br>

What the type is of `amazon` object here?
</div>

In [ ]:
countries[countries.crosses(amazon)]

<div class="alert alert-info" style="font-size:80%">
<b>REFERENCE</b>: <br><br>

<ul>
  <li>`equals`</li>
  <li>`contains`</li>
  <li>`crosses`</li>
  <li>`disjoint`</li>
  <li>`intersects`</li>
  <li>`overlaps`</li>
  <li>`touches`</li>
  <li>`within`</li>
  <li>`covers`</li>
</ul>

<p>
See https://shapely.readthedocs.io/en/stable/manual.html#predicates-and-relationships for an overview of those methods.
<p></p>
See https://en.wikipedia.org/wiki/DE-9IM for all details on the semantics of those operations.
</p>
</div>

<div class="alert alert-success" style="font-size:120%">
<b>Exercise</b>: <br><br>

Does Brazil share a border with any European country?
</div>

In [ ]:
%load 03-geopandas-shapely

In [ ]:
import folium


m = folium.Map()
for k, country in intersec.iterrows():
    gj = folium.GeoJson(country['geometry'])
    folium.Popup(country['name']).add_to(gj)
    gj.add_to(m)

m.fit_bounds(m.get_bounds())

In [ ]:
m

### Rasterio

In [ ]:
import rasterio
from rasterio import plot


fname = path.joinpath('world.rgb.tif')

world = rasterio.open(fname)

type(world)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, (axr, axg, axb) = plt.subplots(1,3, figsize=(12, 4), sharex=True, sharey=True)
plot.show((world, 1), ax=axr, cmap='Reds', title='red channel')
plot.show((world, 2), ax=axg, cmap='Greens', title='green channel')
plot.show((world, 3), ax=axb, cmap='Blues', title='blue channel');

In [ ]:
plot.show_hist(world, bins=50, lw=0.0, stacked=False, alpha=0.3, 
               histtype='stepfilled', title='World Histogram overlaid')

In [ ]:
world.meta

In [ ]:
arr = world.read()

red = arr[0, ...]

In [ ]:
import cartopy
import cartopy.crs as ccrs

fig, ax = plt.subplots(
    figsize=(12, 10),
    subplot_kw={'projection': ccrs.InterruptedGoodeHomolosine()})
ax.set_global()
ax.imshow(red, origin='upper', transform=ccrs.PlateCarree(),
          interpolation=None)
ax.coastlines()
ax.add_feature(cartopy.feature.BORDERS);

### Load really big datasets raster with xarray

In [ ]:
def size(array):
    nbytes = array.nbytes
    if nbytes < (1 << 10):
        size = f'{nbytes} B'
    elif nbytes < (1 << 20):
        size = f'{nbytes / (1 << 10):.02f} KiB'
    elif nbytes < (1 << 30):
        size = f'{nbytes / (1 << 20):.02f} MiB'
    elif nbytes < (1 << 40):
        size = f'{nbytes / (1 << 30):.02f} GiB'
    else:
        size = f'{nbytes / (1 << 40):.02f} TiB'
    return size

In [ ]:
%%time

import xarray as xr


url = ('https://rsignell.s3.amazonaws.com/NLCD/'
       'nlcd_2011_landcover_2011_edition_2014_10_10.tif')

da = xr.open_rasterio(url, chunks={'x':256, 'y':256})

print(f'{size(da)}\n')

In [ ]:
print(da.crs)

In [ ]:
# subset
# top left corner of region of interest
e0, n0 = -93030, 1809990

# size of region in meters
xsize, ysize = 30000, 30000
e1 = e0 + xsize
n1 = n0 - ysize

subset = da[0, ...].sel(x=slice(e0, e1), y=slice(n0, n1))

In [ ]:
subset

In [ ]:
import numpy as np
from rasterio.warp import transform


nx, ny = len(subset['x']), len(subset['y'])
x, y = np.meshgrid(subset['x'], subset['y'])

In [ ]:
lon, lat = transform(
    da.crs,
    {'init': 'EPSG:4326'},
    x.flatten(), y.flatten()
)

lon = np.asarray(lon).reshape((ny, nx))
lat = np.asarray(lat).reshape((ny, nx))
subset.coords['lon'] = (('y', 'x'), lon)
subset.coords['lat'] = (('y', 'x'), lat)

In [ ]:
%%time
fig, ax = plt.subplots(figsize=(10, 10),
                       subplot_kw={'projection': ccrs.PlateCarree()})
# The data will be downloaded only at this step!
subset.plot(ax=ax, x='lon', y='lat', transform=ccrs.PlateCarree(),
            add_colorbar=False)
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = gl.ylabels_right = False

In [ ]:
import folium.plugins


m = folium.Map([37, 0], zoom_start=1, tiles='stamentoner')

bounds = [[lat.min(), lon.min()], [lat.max(), lon.max()]]

folium.raster_layers.ImageOverlay(
    np.array(subset.data),
    bounds=bounds,
    opacity=0.35,
).add_to(m)
m.fit_bounds(m.get_bounds())

In [ ]:
m

# Putting it all together


[http://ioos.github.io/notebooks_demos/code_gallery](http://ioos.github.io/notebooks_demos/code_gallery)

[Hurricane Irma example](http://ioos.github.io/notebooks_demos/notebooks/2017-09-09-hurricane_irma/)

# Packages that are worth checking out:

- hvplot
- geoviews
- gmt-python

In [ ]:
import xarray as xr
import hvplot.xarray

url = 'http://thredds.ucar.edu/thredds/dodsC/grib/FNMOC/WW3/Global_1p0deg/FNMOC_WW3_Global_1p0deg_20180818_0000.grib1'

ds = xr.open_dataset(url)

In [ ]:
ds['sig_wav_ht_surface'].hvplot(groupby='time1', clim=(0, 5))

In [ ]:
from IPython.display import IFrame

src = 'https://ioos.github.io/notebooks_demos/code_gallery'
IFrame(src=src, width=950, height=500)